In [1]:
import numpy as np
import pandas as pd
import math

cgmquantify

In [2]:
#pip install git+git://github.com/brinnaebent/cgmquantify.git
#pip install "cgmquantify"
import cgmquantify as cgm

In [3]:
df = pd.read_csv("/Users/yuxiding/Desktop/dataset.s010", sep="\t")
data = pd.DataFrame()
data['Time'] = df['DisplayTime']
df = df[df.GlucoseValue != 'Low']
data['Glucose'] = pd.to_numeric(df['GlucoseValue'])
data['subjectId'] = df['subjectId']
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%dT%H:%M:%S')
type(data['Time'])
data['Day'] = data['Time'].dt.day
data = data.reset_index()

diabetic = data[data['subjectId'] == "1636-69-091"]
prediabetic = data[data['subjectId'] == "1636-69-032"]
non_diabetic = data[data['subjectId'] == "1636-69-028"]

In [4]:
day = [14,15,16,17,18]
prediabetic = prediabetic[prediabetic['Day'].isin(day)]
prediabetic

prediabetic

writer = pd.ExcelWriter(r'/Users/yuxiding/Desktop/prediabetic.xlsx', engine='xlsxwriter')
prediabetic.to_excel(writer, sheet_name='pre')
writer.save()

In [5]:
df = pd.read_csv("~/Desktop/prediabetic.csv")
data = pd.DataFrame()
data['Time'] = df['Display Time']
df = df[df.GlucoseValue != 'Low']
data['Glucose'] = pd.to_numeric(df['GlucoseValue'])
data['subjectId'] = df['subjectId']
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%dT%H:%M:%S')
type(data['Time'])
data['Day'] = data['Time'].dt.day
data = data.reset_index()
prediabetic = data
prediabetic 

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,index,Time,Glucose,subjectId,Day
0,0,2016-01-14 00:03:15,120,1636-69-032,14
1,1,2016-01-14 00:08:15,120,1636-69-032,14
2,2,2016-01-14 00:13:15,123,1636-69-032,14
3,3,2016-01-14 00:18:15,126,1636-69-032,14
4,4,2016-01-14 00:23:15,127,1636-69-032,14
...,...,...,...,...,...
1434,1434,2016-01-18 23:37:52,136,1636-69-032,18
1435,1435,2016-01-18 23:42:52,126,1636-69-032,18
1436,1436,2016-01-18 23:47:52,132,1636-69-032,18
1437,1437,2016-01-18 23:52:52,135,1636-69-032,18


In [8]:
def interdaycv(df):
    cvx = (np.std(df['Glucose']) / (np.mean(df['Glucose'])))*100
    return cvx

In [9]:
interdaycv(prediabetic)

13.387176917490685

In [10]:
def interdaysd(df):
    interdaysd = np.std(df['Glucose'])
    return interdaysd

In [11]:
interdaysd(prediabetic)

14.322790803934074

In [12]:
def intradaycv(df):
    intradaycv = []
    for i in pd.unique(df['Day']):
        intradaycv.append(interdaycv(df[df['Day']==i]))
    
    intradaycv_mean = np.mean(intradaycv)
    intradaycv_median = np.median(intradaycv)
    intradaycv_sd = np.std(intradaycv)
    
    return intradaycv_mean, intradaycv_median, intradaycv_sd

In [13]:
intradaycv(prediabetic)

(12.381010442444515, 11.294859971428705, 3.712190236384616)

In [15]:
def intradaysd(df):
    intradaysd =[]

    for i in pd.unique(df['Day']):
        intradaysd.append(np.std(df[df['Day']==i]))
    
    intradaysd_mean = np.mean(intradaysd)
    intradaysd_median = np.median(intradaysd)
    intradaysd_sd = np.std(intradaysd)
    return intradaysd_mean, intradaysd_median, intradaysd_sd

In [16]:
intradaysd(prediabetic)

(32.08012199049869, 11.988321783389814, 36.522124573703955)

In [17]:
def TIR(df, sd=1, sr=5):
    up = np.mean(df['Glucose']) + sd*np.std(df['Glucose'])
    dw = np.mean(df['Glucose']) - sd*np.std(df['Glucose'])
    TIR = len(df[(df['Glucose']<= up) & (df['Glucose']>= dw)])*sr 
    return TIR

In [18]:
TIR(prediabetic, sd=1, sr=5)

5025

In [20]:
def TOR(df, sd=1, sr=5):
    up = np.mean(df['Glucose']) + sd*np.std(df['Glucose'])
    dw = np.mean(df['Glucose']) - sd*np.std(df['Glucose'])
    TOR = len(df[(df['Glucose']>= up) | (df['Glucose']<= dw)])*sr
    return TOR

In [21]:
TOR(prediabetic, sd=1, sr=5)

2170

In [22]:
def POR(df, sd=1, sr=5):
    up = np.mean(df['Glucose']) + sd*np.std(df['Glucose'])
    dw = np.mean(df['Glucose']) - sd*np.std(df['Glucose'])
    TOR = len(df[(df['Glucose']>= up) | (df['Glucose']<= dw)])*sr
    POR = (TOR/(len(df)*sr))*100
    return POR

In [23]:
POR(prediabetic, sd=1, sr=5)

30.159833217512162

In [24]:
def MAGE(df, sd=1):
    up = np.mean(df['Glucose']) + sd*np.std(df['Glucose'])
    dw = np.mean(df['Glucose']) - sd*np.std(df['Glucose'])
    MAGE = np.mean(df[(df['Glucose']>= up) | (df['Glucose']<= dw)])
    return MAGE

In [47]:
cgm.MAGE(prediabetic)

index         659.502304
Glucose       107.889401
Day            15.843318
Minfrommid    646.755760
dtype: float64

In [25]:
MAGE(prediabetic, sd=1)

index      659.502304
Glucose    107.889401
Day         15.843318
dtype: float64

In [30]:
def MAGN(df, sd=1):
    up = np.mean(df['Glucose']) + sd*np.std(df['Glucose'])
    dw = np.mean(df['Glucose']) - sd*np.std(df['Glucose'])
    MAGN = np.mean(df[(df['Glucose']<= up) & (df['Glucose']>= dw)])
    return MAGN

In [31]:
MAGN(prediabetic, sd=1)

index      744.693532
Glucose    106.600000
Day         16.065672
dtype: float64

In [32]:
def J_index(df):
    J = 0.001*((np.mean(df['Glucose'])+np.std(df['Glucose']))**2)
    return J

In [33]:
J_index(prediabetic)

14.716521756499343

In [40]:
def LBGI_HBGI(df):
    f = ((np.log(df['Glucose'])**1.084) - 5.381)
    rl = []
    for i in f: 
        if (i <= 0):
            rl.append(22.77*(i**2))
        else:
            rl.append(0)

    LBGI = np.mean(rl)

    rh = []
    for i in f: 
        if (i > 0):
            rh.append(22.77*(i**2))
        else:
            rh.append(0)

    HBGI = np.mean(rh)
    
    return LBGI, HBGI, rh, rl


In [37]:
LBGI_HBGI(prediabetic)

(0.6026933246633753, 0.14138933943859747)

In [41]:
def ADRR(df):
    ADRRl = []
    for i in pd.unique(df['Day']):
        LBGI, HBGI, rh, rl = LBGI_HBGI(df[df['Day']==i])
        LR = np.max(rl)
        HR = np.max(rh)
        ADRRl.append(LR+HR)

    ADRRx = np.mean(ADRRl)
    return ADRRx

In [42]:
ADRR(prediabetic)

7.213358275339132

In [43]:
def MODD(df):
    df['Timefrommidnight'] =  df['Time'].dt.time
    lists=[]
    for i in range(0, len(df['Timefrommidnight'])):
        lists.append(int(df['Timefrommidnight'].iloc[i].strftime('%H:%M:%S')[0:2])*60 + int(df['Timefrommidnight'].iloc[i].strftime('%H:%M:%S')[3:5]) + round(int(df['Timefrommidnight'].iloc[i].strftime('%H:%M:%S')[6:9])/60))
    df['Minfrommid'] = lists
    df = df.drop(columns=['Timefrommidnight'])
    
    #Calculation of MODD and CONGA:
    MODD_n = []
    cgm.uniquetimes = df['Minfrommid'].unique()

    for i in cgm.uniquetimes:
        MODD_n.append(cgm.uniquevalfilter(df, i))
    
    #Remove zeros from dataframe for calculation (in case there are random unique values that result in a mean of 0)
    MODD_n[MODD_n == 0] = np.nan
    
    MODD = np.nanmean(MODD_n)
    return MODD

In [44]:
MODD(prediabetic)

16.292973286875725

In [45]:
def CONGA24(df):
    df['Timefrommidnight'] =  df['Time'].dt.time
    lists=[]
    for i in range(0, len(df['Timefrommidnight'])):
        lists.append(int(df['Timefrommidnight'].iloc[i].strftime('%H:%M:%S')[0:2])*60 + int(df['Timefrommidnight'].iloc[i].strftime('%H:%M:%S')[3:5]) + round(int(df['Timefrommidnight'].iloc[i].strftime('%H:%M:%S')[6:9])/60))
    df['Minfrommid'] = lists
    df = df.drop(columns=['Timefrommidnight'])
    
    #Calculation of MODD and CONGA:
    MODD_n = []
    uniquetimes = df['Minfrommid'].unique()

    for i in cgm.uniquetimes:
        MODD_n.append(cgm.uniquevalfilter(df, i))
    
    #Remove zeros from dataframe for calculation (in case there are random unique values that result in a mean of 0)
    MODD_n[MODD_n == 0] = np.nan
    
    CONGA24 = np.nanstd(MODD_n)
    return CONGA24


CONGA24(prediabetic)

5.902308483766035

In [49]:
def GMI(df):
    GMI = 3.31 + (0.02392*np.mean(df['Glucose']))
    return GMI

In [50]:
GMI(prediabetic)

5.86917403752606

In [51]:
def eA1c(df):
    eA1c = (46.7 + np.mean(df['Glucose']))/ 28.7 
    return eA1c

In [52]:
eA1c(prediabetic)

5.355013281096774

In [53]:
def summary(df):  
    meanG = np.nanmean(df['Glucose'])
    medianG = np.nanmedian(df['Glucose'])
    minG = np.nanmin(df['Glucose'])
    maxG = np.nanmax(df['Glucose'])
    Q1G = np.nanpercentile(df['Glucose'], 25)
    Q3G = np.nanpercentile(df['Glucose'], 75)
    
    return meanG, medianG, minG, maxG, Q1G, Q3G

In [54]:
summary(prediabetic)

(106.98888116747742, 107.0, 67, 156, 97.0, 116.0)

In [68]:
dir(cgm)

['ADRR',
 'CONGA24',
 'GMI',
 'HBGI',
 'J_index',
 'LBGI',
 'LBGI_HBGI',
 'MAGE',
 'MAGN',
 'MODD',
 'POR',
 'TIR',
 'TOR',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__warningregistry__',
 'datetime',
 'eA1c',
 'importdexcom',
 'interdaycv',
 'interdaysd',
 'intradaycv',
 'intradaysd',
 'lowess',
 'np',
 'pd',
 'plotglucosebounds',
 'plotglucosesd',
 'plotglucosesmooth',
 'plt',
 'summary',
 'uniquetimes',
 'uniquevalfilter']

In [135]:
prediabetic

,index,Time,Glucose,subjectId,Day,Timefrommidnight,Minfrommid
5597,5597,2016-01-14 00:03:15,120.0,1636-69-032,14,00:03:15,3
5598,5598,2016-01-14 00:08:15,120.0,1636-69-032,14,00:08:15,8
5599,5599,2016-01-14 00:13:15,123.0,1636-69-032,14,00:13:15,13
5600,5600,2016-01-14 00:18:15,126.0,1636-69-032,14,00:18:15,18
5601,5601,2016-01-14 00:23:15,127.0,1636-69-032,14,00:23:15,23
...,...,...,...,...,...,...,...
7031,7031,2016-01-18 23:37:52,136.0,1636-69-032,18,23:37:52,1418
7032,7032,2016-01-18 23:42:52,126.0,1636-69-032,18,23:42:52,1423
7033,7033,2016-01-18 23:47:52,132.0,1636-69-032,18,23:47:52,1428
7034,7034,2016-01-18 23:52:52,135.0,1636-69-032,18,23:52:52,1433
